In [ ]:
# @tutorial writers: You can remove cells from the HTML/LaTeX output by adding this to the cell metadata:
# "nbsphinx": "hidden"

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# simplify exception output
%xmode Plain

To import the LArray library, run:

In [ ]:
from larray import *

## Axis creation

An [Axis](api.rst#Axis) represents a dimension of an LArray object.
It consists of a name and a list of labels. They are several ways to create an axis:

In [ ]:
# create a wildcard axis 
age = Axis(3, 'age')
# labels given as a list 
time = Axis([2007, 2008, 2009], 'time')
# create an axis using one string
sex = Axis('sex=M,F')
# labels generated using a special syntax  
other = Axis('other=A01..C03')

age, sex, time, other

## Array creation

A [LArray](api.rst#LArray) object represents a multidimensional array with labeled axes.

### From scratch

To create an array from scratch, you need to provide the data and a list
of axes. Optionally, a title can be defined.

In [ ]:
import numpy as np

# list of the axes
axes = [age, sex, time, other]
# data (the shape of data array must match axes lengths)
data = np.random.randint(100, size=[len(axis) for axis in axes])
# title (optional)
title = 'random data'

arr = LArray(data, axes, title)
arr

### Array creation functions

Arrays can also be generated in an easier way through [creation functions](api.rst#array-creation-functions):

-  `ndtest` : creates a test array with increasing numbers as data
-  `empty` : creates an array but leaves its allocated memory
   unchanged (i.e., it contains "garbage". Be careful !)
-  `zeros`: fills an array with 0
-  `ones` : fills an array with 1
-  `full` : fills an array with a given value
-  `sequence` : creates an array from an axis by iteratively applying a function to a given initial value.

Except for ndtest, a list of axes must be provided.
Axes can be passed in different ways:

-  as Axis objects
-  as integers defining the lengths of auto-generated wildcard axes
-  as a string : 'sex=M,F;time=2007,2008,2009' (name is optional)
-  as pairs (name, labels)

Optionally, the type of data stored by the array can be specified using argument dtype.

In [ ]:
# start defines the starting value of data
ndtest(['age=0..2', 'sex=M,F', 'time=2007..2009'], start=-1)

In [ ]:
# start defines the starting value of data
# label_start defines the starting index of labels
ndtest((3, 3), start=-1, label_start=2)

In [ ]:
# empty generates uninitialised array with correct axes 
# (much faster but use with care!).
# This not really random either, it just reuses a portion 
# of memory that is available, with whatever content is there. 
# Use it only if performance matters and make sure all data 
# will be overridden. 
empty(['age=0..2', 'sex=M,F', 'time=2007..2009'])

In [ ]:
# example with anonymous axes
zeros(['0..2', 'M,F', '2007..2009'])

In [ ]:
# dtype=int forces to store int data instead of default float
ones(['age=0..2', 'sex=M,F', 'time=2007..2009'], dtype=int)

In [ ]:
full(['age=0..2', 'sex=M,F', 'time=2007..2009'], 1.23)

All the above functions exist in *(func)_like* variants which take axes from another array

In [ ]:
ones_like(arr)

### Sequence

In [ ]:
# With initial=1.0 and inc=0.5, we generate the sequence 1.0, 1.5, 2.0, 2.5, 3.0, ... 
sequence('sex=M,F', initial=1.0, inc=0.5)

In [ ]:
# With initial=1.0 and mult=2.0, we generate the sequence 1.0, 2.0, 4.0, 8.0, ... 
sequence('age=0..2', initial=1.0, mult=2.0) 

In [ ]:
# Using your own function
sequence('time=2007..2009', initial=2.0, func=lambda value: value**2)

You can also create N-dimensional array by passing (N-1)-dimensional
array to initial, inc or mult argument

In [ ]:
birth = LArray([1.05, 1.15], 'sex=M,F')
cumulate_newborns = sequence('time=2007..2009', initial=0.0, inc=birth)
cumulate_newborns

In [ ]:
initial = LArray([90, 100], 'sex=M,F') 
survival = LArray([0.96, 0.98], 'sex=M,F')
pop = sequence('age=80..83', initial=initial, mult=survival)
pop

## Load/Dump from files

In [ ]:
demography = load_example_data('demography')
household = demography.hh
pop = demography.pop
mortality = demography.qx

### Load from files

Arrays can be loaded from CSV files

```python
# read_tsv is a shortcut when data are separated by tabs instead of commas (default separator of read_csv)
# read_eurostat is a shortcut to read EUROSTAT TSV files  
household = read_csv('hh.csv')
```

or Excel sheets

```python
# loads array from the first sheet if no sheet is given
pop = read_excel('demography.xlsx', 'pop')
```

or HDF5 files (HDF5 is file format designed to store and organize large amounts of data. 
An HDF5 file can contain multiple arrays. 

```python
mortality = read_hdf('demography.h5','qx')
```

See documentation of [reading functions](api.rst#read) for more details

### Dump in files

Arrays can be dumped in CSV files  

```python
household.to_csv('hh2.csv')
```

or in Excel files

```python
# if the file does not already exist, it is created with a single sheet, 
# otherwise a new sheet is added to it
household.to_excel('demography_2.xlsx', overwrite_file=True)
# it is usually better to specify the sheet explicitly (by name or position) though
household.to_excel('demography_2.xlsx', 'hh')
```

or in HDF5 files

```python
household.to_hdf('demography_2.h5', 'hh')
```

See documentation of [writing methods](api.rst#write) for more details

### more Excel IO

#### Write Arrays

Open an Excel file

```python
wb = open_excel('test.xlsx', overwrite_file=True)
```

Put an array in an Excel Sheet, **excluding** headers (labels)

```python
# put arr at A1 in Sheet1, excluding headers (labels)
wb['Sheet1'] = arr
# same but starting at A9
# note that Sheet1 must exist
wb['Sheet1']['A9'] = arr
```

Put an array in an Excel Sheet, **including** headers (labels)

```python
# dump arr at A1 in Sheet2, including headers (labels)
wb['Sheet2'] = arr.dump()
# same but starting at A10
wb['Sheet2']['A10'] = arr.dump()
```

Save file to disk   

```python
wb.save()
```

Close file    

```python
wb.close()
```

#### Read Arrays

Open an Excel file   

```python
wb = open_excel('test.xlsx')
```

Load an array from a sheet (assuming the presence of (correctly formatted) headers and only one array in sheet)

```python
# save one array in Sheet3 (including headers)
wb['Sheet3'] = arr.dump()

# load array from the data starting at A1 in Sheet3
arr = wb['Sheet3'].load()
```

Load an array with its axes information from a range

```python
# if you need to use the same sheet several times,
# you can create a sheet variable
sheet2 = wb['Sheet2']

# load array contained in the 4 x 4 table defined by cells A10 and D14
arr2 = sheet2['A10:D14'].load()
```

#### Read Ranges (experimental)

Load an array (raw data) with no axis information from a range

```python
arr3 = wb['Sheet1']['A1:B4']
```

in fact, this is not really an LArray ...

```python
type(arr3)

larray.io.excel.Range
```

... but it can be used as such  

```python
arr3.sum(axis=0)
```

... and it can be used for other stuff, like setting the formula instead of the value:   

```python
arr3.formula = '=D10+1'
```

In the future, we should also be able to set font name, size, style, etc.   

## Inspecting

In [ ]:
# load population array
pop = load_example_data('demography').pop

Get array summary : dimensions + description of axes

In [ ]:
pop.info

Get axes  

In [ ]:
time, geo, age, sex, nat = pop.axes

Get array dimensions 

In [ ]:
pop.shape

Get number of elements  

In [ ]:
pop.size

Get size in memory

In [ ]:
pop.memory_used

Start viewer (graphical user interface) in read-only mode.
This will open a new window and block execution of the rest of code until the windows is closed! Required PyQt installed.

```python
view(pop)
```

Load array in an Excel sheet

```python
pop.to_excel()
```

## Selection (Subsets)

LArray allows to select a subset of an array either by labels or positions

### Selection by Labels

To take a subset of an array using labels, use brackets [ ].

Let's start by selecting a single element:

In [ ]:
# here we select the value associated with Belgian women 
# of age 50 from Brussels region for the year 2015
pop[2015, 'BruCap', 50, 'F', 'BE']

Continue with selecting a subset using slices and lists of labels

In [ ]:
# here we select the subset associated with Belgian women of age 50, 51 and 52 
# from Brussels region for the years 2010 to 2016
pop[2010:2016, 'BruCap', 50:52, 'F', 'BE']

In [ ]:
# slices bounds are optional: 
# if not given start is assumed to be the first label and stop is the last one.
# Here we select all years starting from 2010
pop[2010:, 'BruCap', 50:52, 'F', 'BE']

In [ ]:
# Slices can also have a step (defaults to 1), to take every Nth labels
# Here we select all even years starting from 2010
pop[2010::2, 'BruCap', 50:52, 'F', 'BE']

In [ ]:
# one can also use list of labels to take non-contiguous labels.
# Here we select years 2008, 2010, 2013 and 2015
pop[[2008, 2010, 2013, 2015], 'BruCap', 50:52, 'F', 'BE']

The order of indexing does not matter either, so you usually do not care/have to remember about axes positions during computation. It only matters for output.

In [ ]:
# order of index doesn't matter
pop['F', 'BE', 'BruCap', [2008, 2010, 2013, 2015], 50:52]

<div class="alert alert-warning">
**Warning:** Selecting by labels as above works well as long as there is no ambiguity.
   When two or more axes have common labels, it may lead to a crash.
   The solution is then to precise to which axis belong the labels.
</div>

In [ ]:
# let us now create an array with the same labels on several axes
age, weight, size = Axis('age=0..80'), Axis('weight=0..120'), Axis('size=0..200')

arr_ws = ndtest([age, weight, size])

In [ ]:
# let's try to select teenagers with size between 1 m 60 and 1 m 65 and weight > 80 kg.
# In this case the subset is ambiguous and this results in an error:
arr_ws[10:18, :80, 160:165]

In [ ]:
# the solution is simple. You need to precise the axes on which you make a selection
arr_ws[age[10:18], weight[:80], size[160:165]]

### Special variable X

When selecting, assiging or using aggregate functions, an axis can be
refered via the special variable ``X``:

-  pop[X.age[:20]]
-  pop.sum(X.age)

This gives you acces to axes of the array you are manipulating. The main
drawback of using **X** is that you lose the autocompletion available from
many editors. It only works with non-wildcard axes.

In [ ]:
# the previous example could have been also written as  
arr_ws[X.age[10:18], X.weight[:80], X.size[160:165]]

### Selection by Positions

Sometimes it is more practical to use positions along the axis, instead of labels. 
You need to add the character ``i`` before the brackets: ``.i[positions]``. 
As for selection with labels, you can use single position or slice or list of positions. 
Positions can be also negative (-1 represent the last element of an axis).

<div class="alert alert-info">
**Note:** Remember that positions (indices) are always **0-based** in Python.
So the first element is at position 0, the second is at position 1, etc.
</div>

In [ ]:
# here we select the subset associated with Belgian women of age 50, 51 and 52 
# from Brussels region for the first 3 years
pop[X.time.i[:3], 'BruCap', 50:52, 'F', 'BE']

In [ ]:
# same but for the last 3 years
pop[X.time.i[-3:], 'BruCap', 50:52, 'F', 'BE']

In [ ]:
# using list of positions
pop[X.time.i[-9,-7,-4,-2], 'BruCap', 50:52, 'F', 'BE']

<div class="alert alert-warning">
**Warning:** The end *indice* (position) is EXCLUSIVE while the end label is INCLUSIVE.
</div>

In [ ]:
# with labels (3 is included)
pop[2015, 'BruCap', X.age[:3], 'F', 'BE']

In [ ]:
# with position (3 is out)
pop[2015, 'BruCap', X.age.i[:3], 'F', 'BE']

You can use ``.i[]`` selection directly on array instead of axes. 
In this context, if you want to select a subset of the first and third axes for example, you must use a full slice ``:`` for the second one.

In [ ]:
# here we select the last year and first 3 ages
# equivalent to: pop.i[-1, :, :3, :, :]
pop.i[-1, :, :3]

### Assigning subsets

#### Assigning value

Assign a value to a subset

In [ ]:
# let's take a smaller array
pop = load_example_data('demography').pop[2016, 'BruCap', 100:105]
pop2 = pop
pop2

In [ ]:
# set all data corresponding to age >= 102 to 0
pop2[102:] = 0
pop2

One very important gotcha though...

<div class="alert alert-warning">
**Warning:** Modifying a slice of an array in-place like we did above should be done with care otherwise you could have **unexpected effects**. The reason is that taking a **slice** subset of an array does not return a copy of that array, but rather a view on that array. To avoid such behavior, use ``.copy()`` method.
</div>
   
Remember:

-  taking a slice subset of an array is extremely fast (no data is
   copied)
-  if one modifies that subset in-place, one also **modifies the
   original array**
-  **.copy()** returns a copy of the subset (takes speed and memory) but
   allows you to change the subset without modifying the original array
   in the same time

In [ ]:
# indeed, data from the original array have also changed
pop

In [ ]:
# the right way
pop = load_example_data('demography').pop[2016, 'BruCap', 100:105]

pop2 = pop.copy()
pop2[102:] = 0
pop2

In [ ]:
# now, data from the original array have not changed this time
pop

#### Assigning Arrays & Broadcasting

Instead of a value, we can also assign an array to a subset. In that
case, that array can have less axes than the target but those which are
present must be compatible with the subset being targeted.

In [ ]:
sex, nat = Axis('sex=M,F'), Axis('nat=BE,FO')
new_value = LArray([[1, -1], [2, -2]],[sex, nat])
new_value

In [ ]:
# this assigns 1, -1 to Belgian, Foreigner men 
# and 2, -2 to Belgian, Foreigner women for all 
# people older than 100
pop[102:] = new_value
pop

<div class="alert alert-warning">
**Warning:** The array being assigned must have compatible axes with the target subset.
</div>

In [ ]:
# assume we define the following array with shape 3 x 2 x 2
new_value = zeros(['age=0..2', sex, nat]) 
new_value

In [ ]:
# now let's try to assign the previous array in a subset with shape 7 x 2 x 2
pop[102:] = new_value

In [ ]:
# but this works
pop[102:104] = new_value
pop

### Boolean filtering

Boolean filtering can be use to extract subsets.

In [ ]:
#Let's focus on population living in Brussels during the year 2016
pop = load_example_data('demography').pop[2016, 'BruCap']

# here we select all males and females with age less than 5 and 10 respectively
subset = pop[((X.sex == 'H') & (X.age <= 5)) | ((X.sex == 'F') & (X.age <= 10))]
subset

<div class="alert alert-info">
**Note:** Be aware that after boolean filtering, several axes may have merged.
</div>

In [ ]:
# 'age' and 'sex' axes have been merged together
subset.info

This may be not what you because previous selections on merged axes are no longer valid

In [ ]:
# now let's try to calculate the proportion of females with age less than 10
subset['F'].sum() / pop['F'].sum()

Therefore, it is sometimes more useful to not select, but rather set to 0 (or another value) non matching elements

In [ ]:
subset = pop.copy()
subset[((X.sex == 'F') & (X.age > 10))] = 0
subset['F', :20]

In [ ]:
# now we can calculate the proportion of females with age less than 10
subset['F'].sum() / pop['F'].sum()

Boolean filtering can also mix axes and arrays. Example above could also have been written as

In [ ]:
age_limit = sequence('sex=M,F', initial=5, inc=5)
age_limit

In [ ]:
age = pop.axes['age']
(age <= age_limit)[:20]

In [ ]:
subset = pop.copy()
subset[X.age > age_limit] = 0
subset['F'].sum() / pop['F'].sum()

Finally, you can choose to filter on data instead of axes

In [ ]:
# let's focus on females older than 90
subset = pop['F', 90:110].copy()
subset

In [ ]:
# here we set to 0 all data < 10
subset[subset < 10] = 0
subset

## Manipulates axes from arrays

In [ ]:
# let's start with
pop = load_example_data('demography').pop[2016, 'BruCap', 90:95]
pop

### Relabeling

Replace all labels of one axis

In [ ]:
# returns a copy by default
pop_new_labels = pop.set_labels('sex', ['Men', 'Women'])
pop_new_labels

In [ ]:
# inplace flag avoids to create a copy
pop.set_labels('sex', ['M', 'F'], inplace=True)

### Renaming axes

Rename one axis

In [ ]:
pop.info

In [ ]:
# 'rename' returns a copy of the array
pop2 = pop.rename('sex', 'gender')
pop2

Rename several axes at once

In [ ]:
# No x. here because sex and nat are keywords and not actual axes
pop2 = pop.rename(sex='gender', nat='nationality')
pop2

### Reordering axes

Axes can be reordered using ``transpose`` method. 
By default, *transpose* reverse axes, otherwise it permutes the axes according to the list given as argument.
Axes not mentioned come after those which are mentioned(and keep their relative order).
Finally, *transpose* returns a copy of the array.

In [ ]:
# starting order : age, sex, nat
pop

In [ ]:
# no argument --> reverse axes
pop.transpose()

# .T is a shortcut for .transpose()
pop.T

In [ ]:
# reorder according to list
pop.transpose('age', 'nat', 'sex')

In [ ]:
# axes not mentioned come after those which are mentioned (and keep their relative order)
pop.transpose('sex')

## Aggregates

Calculate the sum along an axis

In [ ]:
pop = load_example_data('demography').pop[2016, 'BruCap']
pop.sum('age')

or along all axes except one by appending `_by` to the aggregation function

In [ ]:
pop[90:95].sum_by('age')
# is equivalent to 
pop[90:95].sum('sex', 'nat')

There are many other [aggregation functions](api.rst#aggregation-functions):

-  mean, min, max, median, percentile, var (variance), std (standard
   deviation)
-  labelofmin, labelofmax (label indirect minimum/maxium -- labels where the
   value is minimum/maximum)
-  indexofmin, indexofmax (positional indirect minimum/maxium -- position
   along axis where the value is minimum/maximum)
-  cumsum, cumprod (cumulative sum, cumulative product)

## Groups

One can define groups of labels (or indices)

In [ ]:
age = pop.axes['age']

# using indices (remember: 20 will not be included)
teens = age.i[10:20]
# using labels
pensioners = age[67:]
strange = age[[30, 55, 52, 25, 99]]

strange

or rename them

In [ ]:
# method 'named' returns a new group with the given name
teens = teens.named('children')

# operator >> is a shortcut for 'named'
pensioners = pensioners >> 'pensioners'

pensioners 

Then, use them in selections

In [ ]:
pop[strange]

or aggregations

In [ ]:
pop.sum(pensioners)

In [ ]:
# several groups (here you see the interest of groups renaming)
pop.sum((teens, pensioners, strange))

In [ ]:
# combined with other axes
pop.sum((teens, pensioners, strange), 'nat')

## Arithmetic operations

In [ ]:
# go back to our 6 x 2 x 2 example array
pop = load_example_data('demography').pop[2016, 'BruCap', 90:95]
pop

One can do all usual arithmetic operations on an array, it will apply the operation to all elements individually

In [ ]:
# addition
pop + 200

In [ ]:
# multiplication
pop * 2

In [ ]:
# ** means raising to the power (squaring in this case)
pop ** 2

In [ ]:
# % means modulo (aka remainder of division)
pop % 10

More interestingly, it also works between two arrays

In [ ]:
# load mortality equivalent array
mortality = load_example_data('demography').qx[2016, 'BruCap', 90:95] 

# compute number of deaths
death = pop * mortality
death

<div class="alert alert-info">
**Note:** Be careful when mixing different data types.
You can use the method ``astype`` to change the data type of an array.
</div>

In [ ]:
# to be sure to get number of deaths as integers
# one can use .astype() method
death = (pop * mortality).astype(int)
death

But operations between two arrays only works when they have compatible axes (i.e. same labels)

In [ ]:
pop[90:92] * mortality[93:95]

You can override that but at your own risk. 
In that case only the position on the axis is used and not the labels.

In [ ]:
pop[90:92] * mortality[93:95].drop_labels('age')

### Boolean Operations

In [ ]:
pop2 = pop.copy()
pop2['F'] = -pop2['F']
pop2

In [ ]:
# testing for equality is done using == (a single = assigns the value)
pop == pop2

In [ ]:
# testing for inequality
pop != pop2

In [ ]:
# what was our original array like again?
pop

In [ ]:
# & means (boolean array) and
(pop >= 500) & (pop <= 1000)

In [ ]:
# | means (boolean array) or
(pop < 500) | (pop > 1000)

### Arithmetic operations with missing axes

In [ ]:
pop.sum('age')

In [ ]:
# arr has 3 dimensions
pop.info

In [ ]:
# and arr.sum(age) has two
pop.sum('age').info

In [ ]:
# you can do operation with missing axes so this works
pop / pop.sum('age')

### Axis order does not matter much (except for output)

You can do operations between arrays having different axes order.
The axis order of the result is the same as the left array

In [ ]:
pop

In [ ]:
# let us change the order of axes
pop_transposed = pop.T
pop_transposed

In [ ]:
# mind blowing
pop_transposed + pop

## Combining arrays

### Append/Prepend

Append/prepend one element to an axis of an array

In [ ]:
pop = load_example_data('demography').pop[2016, 'BruCap', 90:95] 

# imagine that you have now acces to the number of non-EU foreigners
data = [[25, 54], [15, 33], [12, 28], [11, 37], [5, 21], [7, 19]]
pop_non_eu = LArray(data, pop['FO'].axes)

# you can do something like this
pop = pop.append(nat, pop_non_eu, 'NEU')
pop

In [ ]:
# you can also add something at the start of an axis
pop = pop.prepend('sex', pop.sum('sex'), 'B')
pop

The value being appended/prepended can have missing (or even extra) axes as long as common axes are compatible

In [ ]:
aliens = zeros(pop.axes['sex'])
aliens

In [ ]:
pop = pop.append('nat', aliens, 'AL')
pop

### Extend

Extend an array along an axis with another array *with* that axis (but other labels)

In [ ]:
_pop = load_example_data('demography').pop
pop = _pop[2016, 'BruCap', 90:95] 
pop_next = _pop[2016, 'BruCap', 96:100]

# concatenate along age axis
pop.extend('age', pop_next)

### Stack

Stack several arrays together to create an entirely new dimension

In [ ]:
# imagine you have loaded data for each nationality in different arrays (e.g. loaded from different Excel sheets)
pop_be, pop_fo = pop['BE'], pop['FO']

# first way to stack them
nat = Axis('nat=BE,FO,NEU')
pop = stack([pop_be, pop_fo, pop_non_eu], nat)

# second way
pop = stack([('BE', pop_be), ('FO', pop_fo), ('NEU', pop_non_eu)], 'nat')

pop

## Sorting

Sort an axis (alphabetically if labels are strings)

In [ ]:
pop_sorted = pop.sort_axes('nat')
pop_sorted

Give labels which would sort the axis   

In [ ]:
pop_sorted.labelsofsorted('sex')

Sort according to values   

In [ ]:
pop_sorted.sort_values((90, 'F'))

## Plotting

Create a plot (last axis define the different curves to draw)

In [ ]:
pop.plot()

In [ ]:
# plot total of both sex
pop.sum('sex').plot()

## Interesting methods

In [ ]:
# starting array
pop = load_example_data('demography').pop[2016, 'BruCap', 100:105]
pop

### with total

Add totals to one axis

In [ ]:
pop.with_total('sex', label='B')

Add totals to all axes at once

In [ ]:
# by default label is 'total'
pop.with_total()

### where

where can be used to apply some computation depending on a condition

In [ ]:
# where(condition, value if true, value if false)
where(pop < 10, 0, -pop)

### clip

Set all data between a certain range

In [ ]:
# clip(min, max)
# values below 10 are set to 10 and values above 50 are set to 50
pop.clip(10, 50)

### divnot0

Replace division by 0 to 0

In [ ]:
pop['BE'] / pop['FO']

In [ ]:
# divnot0 replaces results of division by 0 by 0. 
# Using it should be done with care though
# because it can hide a real error in your data.
pop['BE'].divnot0(pop['FO'])

### diff

The ``diff`` method calculates the n-th order discrete difference along a given axis.
The first order difference is given by out[n+1] = in[n + 1] - in[n] along the given axis.

In [ ]:
pop = load_example_data('demography').pop[2005:2015, 'BruCap', 50]
pop

In [ ]:
# calculates 'pop[year+1] - pop[year]'
pop.diff('time')

In [ ]:
# calculates 'pop[year+2] - pop[year]'
pop.diff('time', d=2)

### ratio

In [ ]:
pop.ratio('nat')

# which is equivalent to
pop / pop.sum('nat')

### percents

In [ ]:
# or, if you want the previous ratios in percents
pop.percent('nat')

### growth\_rate

using the same principle than `diff`   

In [ ]:
pop.growth_rate('time')

### shift

The ``shift`` method drops first label of an axis and shifts all subsequent labels

In [ ]:
pop.shift('time')

In [ ]:
# when shift is applied on an (increasing) time axis, 
# it effectively brings "past" data into the future
pop.shift('time').drop_labels('time') == pop[2005:2014].drop_labels('time')

In [ ]:
# this is mostly useful when you want to do operations between the past and now
# as an example, here is an alternative implementation of the .diff method seen above:
pop.i[1:] - pop.shift('time')

### Misc other interesting functions

There are a lot more [interesting functions](api.rst#miscellaneous) available:

- round, floor, ceil, trunc,
- exp, log, log10,
- sqrt, absolute, nan_to_num, isnan, isinf, inverse,
- sin, cos, tan, arcsin, arccos, arctan
- and many many more...

## Sessions

You can group several arrays in a [Session](api.rst#session)

In [ ]:
# load several arrays
arr1, arr2, arr3 = ndtest((3, 3)), ndtest((4, 2)), ndtest((2, 4))

# create and populate a 'session'
s1 = Session()
s1.arr1 = arr1
s1.arr2 = arr2
s1.arr3 = arr3

s1

In [ ]:
s2 = s1.copy()
s3 = s1.copy()

The advantage of sessions is that you can manipulate all of the arrays in them in one shot

```python
# this saves all the arrays in a single excel file (each array on a different sheet)
s1.save('test.xlsx')

# this saves all the arrays in a single HDF5 file (which is a very fast format)
s1.save('test.h5')

# this creates a session out of all arrays in the .h5 file
s2 = Session('test.h5')
```

```python
# this creates a session out of all arrays in the .xlsx file
s3 = Session('test.xlsx')
```

You can compare two sessions

In [ ]:
s1.equals(s2)

In [ ]:
# let us introduce a difference (a variant, or a mistake perhaps)
s2.arr1['a0', 'b1':] = 0

In [ ]:
s1.equals(s2)

In [ ]:
s_diff = s1 != s2
s_diff

This a bit experimental but can be useful nonetheless (open a graphical interface)

```python
compare(s1_diff.arr1, s2_diff.arr1)
```